<a href="https://colab.research.google.com/github/kthink03/KMI-Project_tmp/blob/master/Source/Analysis%20Sentimental/SentimentalPredict_Shipping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from google.colab import files

import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import load_model

In [ ]:
News_Data=pd.read_excel('/content/3. TotalNews.xlsx')
News_Data.columns=['Index','Date','Content'] # 칼럼 명 변경 

In [ ]:
Count=pd.read_excel('/content/2. Count Info.xlsx')
LCount=Count['뉴스 개수'].tolist()

In [ ]:
Content=News_Data['Content'].tolist()

In [ ]:
TotalContent=[] #월 별로 키워드를 append하여 저장 
for i in LCount:
    if len(TotalContent)==0:
        TotalContent.append(Content[0:i])
        pre=i
    else:
        TotalContent.append(Content[pre:pre+i])
        pre+=i

In [ ]:
loaded_model = load_model('/content/model_general_a_no_commonwords.h5')

In [ ]:
X_train=np.load('/content/X_train.npy',allow_pickle=True)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

threshold = 6
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

# 전체 단어 개수 중 빈도수 2이하인 단어 개수는 제거.
# 0번 패딩 토큰과 1번 OOV 토큰을 고려하여 +2
vocab_size = total_cnt - rare_cnt + 2
print('단어 집합의 크기 :',vocab_size)


In [ ]:
tokenizer = Tokenizer(vocab_size, oov_token = 'OOV') 
tokenizer.fit_on_texts(X_train)
max_len=1000

In [ ]:
def sentiment_predict(new_sentence):
    encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩

    pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
    score = float(loaded_model.predict(pad_new)) # 예측

    return score

In [ ]:
List_Pos=[]; List_Neg=[]

for i in tqdm(TotalContent):
    score=sentiment_predict(i)
    List_Pos.append(score); List_Neg.append(1-score)

data={'Pos':List_Pos,'Neg':List_Neg}


In [ ]:
df_score=pd.DataFrame(data,columns=['Pos','Neg'])

In [ ]:
senti=[]
for index,row in df_score.iterrows():
    senti.append(row['Pos']-row['Neg'])

In [ ]:
senti=pd.DataFrame(senti,columns=['Senti'])

In [ ]:
df_total=pd.concat([df_score,senti],axis=1)

In [ ]:
df_total.to_excel('ShippingNewsData_Labeled.xlsx')

files.download('ShippingNewsData_Labeled.xlsx')